In [1]:
import sys
from pathlib import Path

here = Path.cwd().parent.parent
sys.path.append(str(here))

from lion_core.libs import flatten
from typing import Dict, Any
import json


In [2]:
import json
from typing import Any, Dict

# Assume the flatten function is already imported or defined here

def tuple_key_to_str(key):
    if isinstance(key, tuple):
        return ','.join(map(str, key))
    return key

def convert_keys(d):
    if isinstance(d, dict):
        return {tuple_key_to_str(k): convert_keys(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [convert_keys(v) for v in d]
    return d

def run_test(test: Dict[str, Any]) -> bool:
    result = flatten(test['input'], **test.get('kwargs', {}))
    expected = test['expected']
    
    # Convert tuple keys to strings for JSON serialization
    json_safe_result = convert_keys(result)
    json_safe_expected = convert_keys(expected)
    
    passed = result == expected
    print(f"Test: {test['name']}")
    print(f"Input: {test['input']}")
    print(f"Expected: {json_safe_expected}")
    print(f"Result: {json_safe_result}")
    print(f"Passed: {passed}")
    print("--------------------")
    return passed

# Expanded test cases
test_cases = [
    {
        "name": "Simple dictionary (default behavior)",
        "input": {"a": 1, "b": 2, "c": 3},
        "expected": {("a",): 1, ("b",): 2, ("c",): 3},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Nested dictionary (default behavior)",
        "input": {"a": 1, "b": {"c": 2, "d": 3}},
        "expected": {("a",): 1, ("b", "c"): 2, ("b", "d"): 3},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Nested dictionary with coerce_keys",
        "input": {"a": 1, "b": {"c": 2, "d": 3}},
        "expected": {"a": 1, "b|c": 2, "b|d": 3},
        "kwargs": {}
    },
    {
        "name": "List in dictionary (default behavior)",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {("a",): [1, 2, 3], ("b",): 4},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "List in dictionary with dynamic",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {("a", "0"): 1, ("a", "1"): 2, ("a", "2"): 3, ("b",): 4},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "List in dictionary with dynamic and coerce_keys",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {"a|0": 1, "a|1": 2, "a|2": 3, "b": 4},
        "kwargs": {"dynamic": True}
    },
    {
        "name": "Deeply nested structure",
        "input": {"a": {"b": {"c": {"d": 1}}}},
        "expected": {("a", "b", "c", "d"): 1},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Deeply nested structure with coerce_keys",
        "input": {"a": {"b": {"c": {"d": 1}}}},
        "expected": {"a|b|c|d": 1},
        "kwargs": {}
    },
    {
        "name": "Mixed types with dynamic",
        "input": {"a": 1, "b": "string", "c": [1, 2], "d": {"e": 3}},
        "expected": {("a",): 1, ("b",): "string", ("c", "0"): 1, ("c", "1"): 2, ("d", "e"): 3},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Tuple as input",
        "input": (1, 2, 3),
        "expected": {("0",): 1, ("1",): 2, ("2",): 3},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Coerce sequence to list",
        "input": {"a": (1, 2, 3), "b": 4},
        "expected": {("a", "0"): 1, ("a", "1"): 2, ("a", "2"): 3, ("b",): 4},
        "kwargs": {"dynamic": True, "coerce_sequence_to_list": True, "coerce_keys": False}
    },
    {
        "name": "Coerce sequence to dict",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {("a", "0"): 1, ("a", "1"): 2, ("a", "2"): 3, ("b",): 4},
        "kwargs": {"dynamic": True, "coerce_sequence_to_dict": True, "coerce_keys": False}
    },
    {
        "name": "Max depth 1",
        "input": {"a": {"b": {"c": 1}}, "d": [1, 2, 3]},
        "expected": {("a",): {"b": {"c": 1}}, ("d",): [1, 2, 3]},
        "kwargs": {"max_depth": 1, "coerce_keys": False}
    },
    {
        "name": "Max depth 2 with dynamic",
        "input": {"a": {"b": {"c": 1}}, "d": [1, 2, [3, 4]]},
        "expected": {("a", "b"): {"c": 1}, ("d", "0"): 1, ("d", "1"): 2, ("d", "2"): [3, 4]},
        "kwargs": {"max_depth": 2, "dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Custom separator",
        "input": {"a": {"b": 1}},
        "expected": {"a___b": 1},
        "kwargs": {"sep": "___"}
    },
    {
        "name": "Empty input",
        "input": {},
        "expected": {},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Non-dict input (list)",
        "input": [1, 2, 3],
        "expected": {("0",): 1, ("1",): 2, ("2",): 3},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Nested lists with dynamic",
        "input": [1, [2, 3, [4, 5]], 6],
        "expected": {("0",): 1, ("1", "0"): 2, ("1", "1"): 3, ("1", "2", "0"): 4, ("1", "2", "1"): 5, ("2",): 6},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Dictionary with None values",
        "input": {"a": None, "b": {"c": None}},
        "expected": {("a",): None, ("b", "c"): None},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Dictionary with empty structures",
        "input": {"a": {}, "b": [], "c": ""},
        "expected": {("a",): {}, ("b",): [], ("c",): ""},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Simple dictionary (default behavior)",
        "input": {"a": 1, "b": 2, "c": 3},
        "expected": {("a",): 1, ("b",): 2, ("c",): 3},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Nested dictionary (default behavior)",
        "input": {"a": 1, "b": {"c": 2, "d": 3}},
        "expected": {("a",): 1, ("b", "c"): 2, ("b", "d"): 3},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Nested dictionary with coerce_keys",
        "input": {"a": 1, "b": {"c": 2, "d": 3}},
        "expected": {"a": 1, "b|c": 2, "b|d": 3},
        "kwargs": {}
    },
    {
        "name": "List in dictionary (default behavior)",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {("a",): [1, 2, 3], ("b",): 4},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "List in dictionary with dynamic",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {("a", "0"): 1, ("a", "1"): 2, ("a", "2"): 3, ("b",): 4},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "List in dictionary with dynamic and coerce_keys",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {"a|0": 1, "a|1": 2, "a|2": 3, "b": 4},
        "kwargs": {"dynamic": True}
    },
    {
        "name": "Deeply nested structure",
        "input": {"a": {"b": {"c": {"d": 1}}}},
        "expected": {("a", "b", "c", "d"): 1},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Deeply nested structure with coerce_keys",
        "input": {"a": {"b": {"c": {"d": 1}}}},
        "expected": {"a|b|c|d": 1},
        "kwargs": {}
    },
    {
        "name": "Mixed types with dynamic",
        "input": {"a": 1, "b": "string", "c": [1, 2], "d": {"e": 3}},
        "expected": {("a",): 1, ("b",): "string", ("c", "0"): 1, ("c", "1"): 2, ("d", "e"): 3},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Tuple as input",
        "input": (1, 2, 3),
        "expected": {("0",): 1, ("1",): 2, ("2",): 3},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Coerce sequence to list",
        "input": {"a": (1, 2, 3), "b": 4},
        "expected": {("a", "0"): 1, ("a", "1"): 2, ("a", "2"): 3, ("b",): 4},
        "kwargs": {"dynamic": True, "coerce_sequence_to_list": True, "coerce_keys": False}
    },
    {
        "name": "Coerce sequence to dict",
        "input": {"a": [1, 2, 3], "b": 4},
        "expected": {("a", "0"): 1, ("a", "1"): 2, ("a", "2"): 3, ("b",): 4},
        "kwargs": {"dynamic": True, "coerce_sequence_to_dict": True, "coerce_keys": False}
    },
    {
        "name": "Max depth 1",
        "input": {"a": {"b": {"c": 1}}, "d": [1, 2, 3]},
        "expected": {("a",): {"b": {"c": 1}}, ("d",): [1, 2, 3]},
        "kwargs": {"max_depth": 1, "coerce_keys": False}
    },
    {
        "name": "Max depth 2 with dynamic",
        "input": {"a": {"b": {"c": 1}}, "d": [1, 2, [3, 4]]},
        "expected": {("a", "b"): {"c": 1}, ("d", "0"): 1, ("d", "1"): 2, ("d", "2"): [3, 4]},
        "kwargs": {"max_depth": 2, "dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Custom separator",
        "input": {"a": {"b": 1}},
        "expected": {"a___b": 1},
        "kwargs": {"sep": "___"}
    },
    {
        "name": "Empty input",
        "input": {},
        "expected": {},
        "kwargs": {}
    },
    {
        "name": "Non-dict input (list)",
        "input": [1, 2, 3],
        "expected": {("0",): 1, ("1",): 2, ("2",): 3},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Nested lists with dynamic",
        "input": [1, [2, 3, [4, 5]], 6],
        "expected": {("0",): 1, ("1", "0"): 2, ("1", "1"): 3, ("1", "2", "0"): 4, ("1", "2", "1"): 5, ("2",): 6},
        "kwargs": {"dynamic": True, "coerce_keys": False}
    },
    {
        "name": "Dictionary with None values",
        "input": {"a": None, "b": {"c": None}},
        "expected": {("a",): None, ("b", "c"): None},
        "kwargs": {"coerce_keys": False}
    },
    {
        "name": "Dictionary with empty structures",
        "input": {"a": {}, "b": [], "c": ""},
        "expected": {("a",): {}, ("b",): [], ("c",): ""},
        "kwargs": {"coerce_keys": False}
    }
]

# Run all tests
passed_tests = sum(run_test(test) for test in test_cases)
total_tests = len(test_cases)

print(f"Passed {passed_tests} out of {total_tests} tests.")

Test: Simple dictionary (default behavior)
Input: {'a': 1, 'b': 2, 'c': 3}
Expected: {'a': 1, 'b': 2, 'c': 3}
Result: {'a': 1, 'b': 2, 'c': 3}
Passed: True
--------------------
Test: Nested dictionary (default behavior)
Input: {'a': 1, 'b': {'c': 2, 'd': 3}}
Expected: {'a': 1, 'b,c': 2, 'b,d': 3}
Result: {'a': 1, 'b,c': 2, 'b,d': 3}
Passed: True
--------------------
Test: Nested dictionary with coerce_keys
Input: {'a': 1, 'b': {'c': 2, 'd': 3}}
Expected: {'a': 1, 'b|c': 2, 'b|d': 3}
Result: {'a': 1, 'b|c': 2, 'b|d': 3}
Passed: True
--------------------
Test: List in dictionary (default behavior)
Input: {'a': [1, 2, 3], 'b': 4}
Expected: {'a': [1, 2, 3], 'b': 4}
Result: {'b': 4, 'a': [1, 2, 3]}
Passed: True
--------------------
Test: List in dictionary with dynamic
Input: {'a': [1, 2, 3], 'b': 4}
Expected: {'a,0': 1, 'a,1': 2, 'a,2': 3, 'b': 4}
Result: {'b': 4, 'a,0': 1, 'a,1': 2, 'a,2': 3}
Passed: True
--------------------
Test: List in dictionary with dynamic and coerce_keys
Input: {'